In [ ]:
# ===============================
# Step 0: Install & Import Libraries
# ===============================
# pip install transformers torch pandas scikit-learn
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:86: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


ImportError: cannot import name 'AdamW' from 'transformers' (/usr/local/lib/python3.12/dist-packages/transformers/__init__.py)

In [ ]:
# ===============================
# Step 1: Generate Dataset
# ===============================
# Ensure necessary imports present when running this cell standalone
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
sentences = [
    "I love this movie", "This film was amazing", "I enjoyed every moment",
    "The acting was great", "What a fantastic experience",
    "Absolutely wonderful movie", "The story was touching", "I liked the characters",
    "Very entertaining film", "This movie made me happy",
    
    "I hate this movie", "This film was terrible", "I disliked every moment",
    "The acting was awful", "What a boring experience",
    "Absolutely horrible movie", "The story was weak", "I hated the characters",
    "Very disappointing film", "This movie made me angry"
] * 5  # 100 samples

labels = ["positive"] * 50 + ["negative"] * 50
df = pd.DataFrame({"sentence": sentences, "label": labels})

# ===============================
# Step 2: Encode Labels
# ===============================
label_encoder = LabelEncoder()
df["label_encoded"] = label_encoder.fit_transform(df["label"])  # positive=1, negative=0

# ===============================
# Step 3: Train-Test Split
# ===============================
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["sentence"], df["label_encoded"], test_size=0.2, random_state=42
)

# ===============================
# Step 4: Load GPT-2 Tokenizer & Model
# ===============================
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# GPT-2 has no pad token by default
tokenizer.pad_token = tokenizer.eos_token

model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=2)
model.config.pad_token_id = model.config.eos_token_id

# Use GPU if available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# ===============================
# Step 5: Tokenize Dataset
# ===============================
max_len = 32  # max tokens per sentence

def tokenize_texts(texts):
    return tokenizer(
        list(texts),
        padding=True,
        truncation=True,
        max_length=max_len,
        return_tensors="pt"
    )

train_encodings = tokenize_texts(train_texts)
test_encodings = tokenize_texts(test_texts)

# ===============================
# Step 6: Create PyTorch Dataset
# ===============================
class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

train_dataset = SentimentDataset(train_encodings, list(train_labels))
test_dataset = SentimentDataset(test_encodings, list(test_labels))

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# ===============================
# Step 7: Optimizer
# ===============================
optimizer = AdamW(model.parameters(), lr=5e-5)

# ===============================
# Step 8: Training Loop
# ===============================
model.train()
epochs = 3

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
    print(f"Loss: {loss.item():.4f}")

# ===============================
# Step 9: Evaluation
# ===============================
model.eval()
preds = []
true_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        
        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=-1)
        
        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(true_labels, preds)
print(f"Test Accuracy: {accuracy:.4f}")

# ===============================
# Step 10: Predict New Sentences
# ===============================
def predict_sentiment(text):
    model.eval()
    encoding = tokenizer(text, truncation=True, padding=True, max_length=max_len, return_tensors="pt")
    encoding = {key: val.to(device) for key, val in encoding.items()}
    with torch.no_grad():
        output = model(**encoding)
        prediction = torch.argmax(output.logits, dim=-1).item()
    return "Positive" if prediction == 1 else "Negative"

# Test
print(predict_sentiment("I really enjoyed this movie"))
print(predict_sentiment("This film was very boring"))


ImportError: cannot import name 'AdamW' from 'transformers' (/usr/local/lib/python3.12/dist-packages/transformers/__init__.py)